# Paths

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [2]:
model_path = "/data/home/milinbhade/LLaMa/hf_llama/"
hf_llama_chat_path = "/data/home/milinbhade/LLaMa/hf_llama_chat/"

datapath = "/data/home/milinbhade/LLaMa/pretrain_data/alpaca/alpaca-cleaned-dataset"

redpajama_datapath = "./redpajama/"

In [3]:
import torch
torch.cuda.device_count()

1

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.is_bf16_supported())

True
False


In [5]:
# import logging
# import os
# from contextlib import nullcontext

# TRL_USE_RICH = os.environ.get("TRL_USE_RICH", False)

# from trl.commands.cli_utils import init_zero_verbose, SFTScriptArguments, TrlParser

# if TRL_USE_RICH:
#     init_zero_verbose()
#     FORMAT = "%(message)s"

#     from rich.console import Console
#     from rich.logging import RichHandler

# import torch
# from datasets import load_dataset

# from tqdm.rich import tqdm
# from transformers import AutoTokenizer

# from trl import (
#     ModelConfig,
#     RichProgressCallback,
#     SFTConfig,
#     SFTTrainer,
#     get_peft_config,
#     get_quantization_config,
#     get_kbit_device_map,
# )


# Imports

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# from trl import (
#     ModelConfig,
#     RichProgressCallback,
#     SFTConfig,
#     SFTTrainer,
#     get_peft_config,
#     get_quantization_config,
#     get_kbit_device_map,
# )

2024-05-29 20:24:20.572080: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 20:24:21.624322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
dataset = load_dataset("togethercomputer/RedPajama-Data-1T-Sample", cache_dir=redpajama_datapath)

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for togethercomputer/RedPajama-Data-1T-Sample contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/togethercomputer/RedPajama-Data-1T-Sample
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [8]:
print(dataset['train'][100]['text'])

\section{Introduction}

Members of our group have been involved in long-term studies of abundances in  
Galactic halo stars. 
These studies have been designed to address a number of important 
issues, including: the synthesis mechanisms of 
the heavy,
specifically, neutron capture ($n$-capture) 
elements, early in the history of the
Galaxy; the identities of the earliest stellar generations,
the progenitors of the
halo stars; the site or sites for the synthesis of the 
rapid $n$-capture ({\it i.e.}, $r$-process) material
throughout the Galaxy; the Galactic Chemical Evolution (GCE) of
the elements; and by employing the abundances of the
radioactive elements (Th and U) as 
chronometers, the ages of the oldest stars, and hence the lower limit
on the age of the Galaxy and the Universe. (See \citealt{truran02},
\citealt{snedencowan03}, \citealt{cowan04}, and \citealt{sneden08}
for discussions of these related and significant topics.)  

In the following paper we review some of the results o

In [9]:
import transformers
from transformers import LlamaTokenizer, AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left", padding=True, truncation=True)
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_path,
#     tokenizer=tokenizer,
#     torch_dtype=torch.float16,
#     trust_remote_code=True,
#     device_map="auto",
#     load_in_8bit=True,
#     return_full_text=False
# )

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=True,
    device_map="auto",
)

In [ ]:
# result = pipeline(['Suppose I go in the east direction, what'])
# print(result[0])

In [ ]:
dataset = dataset['train']

In [ ]:
from transformers import LlamaConfig

In [ ]:
model_config = LlamaConfig.from_pretrained(model_path)

In [ ]:
from peft import LoraConfig

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
model

<!-- parser = TrlParser((SFTScriptArguments, SFTConfig, ModelConfig))
args, training_args, model_config = parser.parse_args_and_config() -->

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    set_seed,
)

from trl import SFTTrainer

In [ ]:
output_dir="./lora_alpaca"

In [ ]:
args = transformers.TrainingArguments(
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    warmup_steps=10,
    num_train_epochs=3,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=10,
    logging_first_step=True,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=100,
    save_steps=200,
    output_dir=output_dir,
    save_total_limit=20,
    load_best_model_at_end=True,
    ddp_find_unused_parameters=None,
    group_by_length=False,
    report_to="wandb",
    run_name=output_dir.split('/')[-1],
    metric_for_best_model="{}_loss".format(redpajama_datapath),
        )

# training_args = TrainingArguments(
#         output_dir=args.output_dir,
#         dataloader_drop_last=True,
#         eval_strategy="steps",
#         max_steps=args.max_steps,
#         eval_steps=args.eval_freq,
#         save_steps=args.save_freq,
#         logging_steps=args.log_freq,
#         per_device_train_batch_size=args.batch_size,
#         per_device_eval_batch_size=args.batch_size,
#         learning_rate=args.learning_rate,
#         lr_scheduler_type=args.lr_scheduler_type,
#         warmup_steps=args.num_warmup_steps,
#         gradient_accumulation_steps=args.gradient_accumulation_steps,
#         gradient_checkpointing=args.gradient_checkpointing,
#         fp16=args.fp16,
#         bf16=args.bf16,
#         weight_decay=args.weight_decay,
#         run_name="llama-7b-finetuned",
#         report_to="wandb",
#         ddp_find_unused_parameters=False,
#     )

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    args=args,
    peft_config=peft_config,
    packing=True,
)


In [ ]:
trainer.train()

# Code

In [10]:
from peft import LoraConfig
from transformers import BitsAndBytesConfig

loraConfig = LoraConfig(
    r = 8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [12]:
from transformers import TrainingArguments

trainArgs = TrainingArguments(
    report_to="wandb",
    output_dir="devfiles/results/run2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=5,
    logging_steps=1,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=25,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    gradient_checkpointing=True,
    fp16=False,
    bf16=False)

In [13]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    args = trainArgs,
    train_dataset = dataset['train'],
    peft_config = loraConfig,
    packing = True,
    dataset_text_field = "text",
    tokenizer = tokenizer
)


/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': packing, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)


AttributeError: 'TrainingArguments' object has no attribute 'model_init_kwargs'

In [ ]:
trainer.train()